In [8]:
#drop cols ur not aggregating by
#extra case statements depneding on granularity (don't worry about making function global for every date granularity)

In [9]:
#data stuff
import time
import pandas as pd
import numpy as np
import datetime as dt

#regression stuff
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

#graph stuff
import seaborn as sns
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import plotly
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'


#import functions from other file that we want
from utilities import *

# 0.1 Read in data

In [163]:
#initial load of clickthru data from vw dataset
q = '''SELECT 
     account
    , attr_window
    , campaign
    , _match
    , campaign_objective
    , Year
    , Quarter
    , Month
    , week
    , week_start
    , date
    , marketing_initiative
    , marketing_segment
    , marketing_subinitiative
    , channel
    , platform
    , site
    , sum(case when sor_prod = 'All (Core All, NPV)' then spend else 0 end) spend
    , sum(case when sor_prod = 'All (Core All, NPV)' then clicks else 0 end) clicks
    , sum(case when sor_prod = 'All (Core All, NPV)' then impressions else 0 end) impressions
    , sum(case
            when sor_prod = 'Core' then conversions else null end) digi_ada_conversions
    , sum(case
            when sor_prod = 'Crosswords' then conversions else null end) games_conversions
    , sum(case
            when sor_prod = 'EDU' then conversions else null end) edu_conversions
    , sum(case
            when sor_prod = 'Home Delivery' then conversions else null end) hd_conversions
    , sum(case
            when sor_prod = 'CK' then conversions else null end) ck_conversions     
    , sum(case
            when sor_prod = 'Core All (Core, HD, EDU)' then conversions else null end) core_conversions
    , sum(case
            when sor_prod = 'All (Core All, NPV)' then conversions else null end) all_conversions
FROM `nyt-mkt-prd.paid_media_data.placement_daily_vw`
WHERE date >= '2020-01-01' and attr_window in('Media Reported - Click Thru')
group by 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12,13,14,15,16,17
order by date, campaign, _match
    '''
start_time = time.time() 

clickthru_df = pd.read_gbq(q,
                 project_id ='nyt-bigquery-beta-workspace',
                 dialect='standard')

print(f'time took: {str(round(time.time() - start_time, 2))}')

time took: 208.06


In [164]:
clickthru_df.shape

(884207, 27)

In [165]:
#initial load of viewthru data from vw dataset
q = '''SELECT 
     account
    , attr_window
    , campaign
    , _match
    , campaign_objective
    , Year
    , Quarter
    , Month
    , week
    , week_start
    , date
    , marketing_initiative
    , marketing_segment
    , marketing_subinitiative
    , channel
    , platform
    , site
    , sum(case when sor_prod = 'All (Core All, NPV)' then spend else 0 end) spend
    , sum(case when sor_prod = 'All (Core All, NPV)' then clicks else 0 end) clicks
    , sum(case when sor_prod = 'All (Core All, NPV)' then impressions else 0 end) impressions
    , sum(case
            when sor_prod = 'Core' then conversions else null end) digi_ada_conversions
    , sum(case
            when sor_prod = 'Crosswords' then conversions else null end) games_conversions
    , sum(case
            when sor_prod = 'EDU' then conversions else null end) edu_conversions
    , sum(case
            when sor_prod = 'Home Delivery' then conversions else null end) hd_conversions
    , sum(case
            when sor_prod = 'CK' then conversions else null end) ck_conversions     
    , sum(case
            when sor_prod = 'Core All (Core, HD, EDU)' then conversions else null end) core_conversions
    , sum(case
            when sor_prod = 'All (Core All, NPV)' then conversions else null end) all_conversions
FROM `nyt-mkt-prd.paid_media_data.placement_daily_vw`
WHERE date >= '2020-01-01' and attr_window in('Media Reported - View Thru')
group by 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12,13,14,15,16,17
order by date, campaign, _match
    '''
start_time = time.time() 

viewthru_df = pd.read_gbq(q,
                 project_id ='nyt-bigquery-beta-workspace',
                 dialect='standard')

print(f'time took: {str(round(time.time() - start_time, 2))}')

time took: 204.67


In [166]:
#rename columns in the separate dataframes to identify clickthru and viewthru
#first clickthru
for col in clickthru_df.columns:
    if 'conversions' in col: 
        clickthru_df[col+'_ct'] = clickthru_df[col]
        clickthru_df.drop(col, axis=1, inplace=True)
        
#now viewthru
for col in viewthru_df.columns:
    if 'conversions' in col: 
        viewthru_df[col+'_vt'] = viewthru_df[col]
        viewthru_df.drop(col, axis=1, inplace=True)

In [167]:
#now merge the dataframes 
#create a truncated version of viewthru DF to make the merge more efficient
vt_merge = viewthru_df[['account','campaign','_match','date','marketing_initiative','marketing_segment'] + [col for col in viewthru_df.columns if 'conversions' in col]]

#merge with clickthru df
df = clickthru_df.merge(vt_merge, how='left', on=[col for col in vt_merge.columns if 'conversions' not in col])

In [168]:
#make a copy
df1 = df

In [169]:
#d1 -> save read in results up to 7/13/21
df1.to_pickle("df_raw.pkl")
df1 = pd.read_pickle("df_raw.pkl")

In [170]:
#create table
df.to_gbq(destination_table = 'bollinger_data.view_through_raw_data',
          project_id = 'nyt-bigquery-beta-workspace',if_exists = 'replace')

In [171]:
#read in from new, static table
q = '''SELECT 
     *
FROM `nyt-bigquery-beta-workspace.bollinger_data.view_through_raw_data` 
WHERE date between '2020-01-01' and '2021-06-30'
    '''
df = pd.read_gbq(q,
                 project_id ='nyt-bigquery-beta-workspace',
                 dialect='standard')
print(f'time took: {str(round(time.time() - start_time, 2))}')

time took: 1785.85


In [406]:
df.shape

(622756, 41)

# 0.2 Data Cleaning

## Data Removal

In [173]:
#exclude Paid Search 
df = df.loc[df['channel'] != 'Paid Search']

In [175]:
#exclude Brand, Branding, Retention, App Download, and App Install initatives bc not relevant to Chrome cookie loss
df = df[~df['marketing_initiative'].isin(['Brand','Branding','Retention','App Download', 'App-Install'])]

In [177]:
#exclude where spend = 0 but impressions, clicks, or conversions are not equal to 0
#~ wrap in parens 

#35,000 single digit

df = df.loc[~((df['spend'] == 0) & 
(
                            (df['impressions'] > 100) |
                            (df['clicks'] > 100) |
                            (df['all_conversions_vt'] > 100) | 
                            (df['all_conversions_ct'] > 100)
             ))]

In [466]:
#exclude media blackout -> May 14,2020 through June 30,2020
df = df[(df['date'] < '2020-05-01') | (df['date'] >= '2020-07-01')]
df.shape

(617652, 41)

In [407]:
# outliers -> perhaps revisit after error to accommodate for news cycle but rn don't remove

## Impute X-UNKNOWN-X

In [181]:
#replace all NaNs to 0 
df = df.fillna(0)

In [182]:
df['marketing_subinitiative'].value_counts()

Core             464926
Games             46245
Cooking           42290
Home Delivery     40067
X-UNKNOWN-X       27021
EDU                2057
Audio               150
Name: marketing_subinitiative, dtype: int64

In [183]:
#Twigeo campaigns missing
df.loc[(df['marketing_subinitiative'] == 'X-UNKNOWN-X') & 
       (
          (df['campaign'].str.contains('game')) | 
          (df['campaign'].str.contains('xwd')) | 
          (df['campaign'].str.contains('cross'))
       ), 'marketing_subinitiative'] = 'Games (former: Crosswords)'

In [184]:
#same for cooking
df.loc[(df['marketing_subinitiative'] == 'X-UNKNOWN-X') & 
       (
          (df['campaign'].str.contains('cook')) | 
          (df['campaign'].str.contains('_ck'))
       ), 'marketing_subinitiative'] = 'Cooking'

## Low-Volume Grouping

In [185]:
#group low-volume channels 
df['new_channel'] = df['channel']
df.loc[~df['channel'].isin(['Social','Display','Paid Search','Native']), 'new_channel'] = 'Low-Volume Channels'

In [186]:
#group low-volume sites
df['new_site'] = df['site']
df.loc[~df['site'].isin(['The Trade Desk','DBM_US','Snap','Facebook','Twitter','Google AdWords (DSA)','Bing Ads', 'Google AdWords']), 'new_site'] = 'Low-Volume Sites'

In [187]:
#group low-volume products
df['new_product'] = df['marketing_subinitiative']
df.loc[~df['marketing_subinitiative'].isin(['Home Delivery','Core','Games','Cooking']), 'new_product'] = 'Low-Volume Products'

In [188]:
#check to see if the above groupings work
df['channel'].value_counts()
df['new_channel'].value_counts()
df['site'].value_counts()
df['new_site'].value_counts()
df['marketing_subinitiative'].value_counts()
df['new_product'].value_counts()

Core                   464926
Games                   46245
Cooking                 42290
Home Delivery           40067
Low-Volume Products     29228
Name: new_product, dtype: int64

## Adding, Dropping, and Renaming Columns

In [189]:
#group dates by bigger cols using the dt module 
#apply() -> so it reads better in graphs 

#create date-based aggregation columns for easier grouping
df['year_month_cal'] = df['date'].dt.to_period('M').apply(lambda r: r.start_time)
df['year_week_monday'] = df['date'].dt.to_period('W').apply(lambda r: r.start_time)
df['year_quarter'] = df['date'].dt.to_period('Q').apply(lambda r: r.start_time)

/Users/chloebollinger/.local/share/virtualenvs/python_3_8_test_env-uedWa5dM/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [190]:
#create a composite channel-marketing-initiative column for easier grouping by combos of channel and marketing_initiative 
df['channel_mkt_init'] = df['channel'] + " - " + df['marketing_initiative']

In [191]:
#get rid of attr_window (bc it isn't needed anymore just says click thru)
df = df.drop(['attr_window'], axis=1)

In [192]:
#add total conversions column
df['total_conversions'] = df['all_conversions_ct'] + df['all_conversions_vt']

# 0.3 Create Dataframes Split By Channel

In [409]:
df['new_channel'].value_counts()

Display                244309
Native                 233033
Social                 101599
Low-Volume Channels     43815
Name: new_channel, dtype: int64

In [410]:
df_low_channel_volume = df[df['new_channel'] == 'Low-Volume Channels']
df_low_channel_volume.shape

(43815, 41)

In [411]:
df_display = df[df['channel'] == 'Display']
df_display.shape

(244309, 41)

In [412]:
df_native = df[df['channel'] == 'Native']
df_native.shape

(233033, 41)

In [413]:
df_display_native = df[(df['channel'] == 'Display') | (df['channel'] == 'Native')]
df_display_native.shape

(477342, 41)

In [414]:
df_social = df[df['channel'] == 'Social']
df_social.shape

(101599, 41)

In [415]:
df_social['site'].value_counts()

Facebook       59900
Snap           35737
Twitter         5884
LinkedIn US       60
Pinterest         16
Reddit             2
Name: site, dtype: int64

In [416]:
df_fb = df_social[df_social['site'] == 'Facebook']
df_fb.shape

(59900, 41)

In [417]:
df_snap = df_social[df_social['site'] == 'Snap']
df_snap.shape

(35737, 41)

In [418]:
df_twitter= df_social[df_social['site'] == 'Twitter']
df_twitter.shape

(5884, 41)

# 1. Functions

## 1.1 Multiplier Functions

In [419]:
'''
    purpose
    --------
        Calculate a multiplier that uses click through conversions to project view through conversions.
        The multiplier calculates vt conversions using ct conversions.
    arguments
    ---------
        df : dataframe
            raw BQ data formatted based on the above query 
        date_granularity : str (exact column name)
            level of granularity when grouping the data to calculate a multiplier 
        start_date : datetime
            training start date (default 2020-01-01)
        end_date : datetime
            training end date (default 2020-12-31)
    returns
    -------
        df_copy: dataframe
            dataframe with multiplier 
            [0]
        df_copy['indiv_multiplier'].mean() : int
            the multiplier number itself 
            [1]
'''
#NOTES TO RESOLVE: if any multipliers are 0 -> manually change to 1 (we'll still get 0 -> 0/1)

def calculate_multiplier(df,date_granularity,start_date='2020-01-01',end_date='2020-12-31'):
    #create a copy of df and filter it to date args
    df_copy = df
    df_copy= df_copy[(df_copy['date'] >= start_date) & (df_copy['date'] <= end_date)]
    
    #sum each col, grouped by desired granularity
    df_copy = df_copy.groupby([date_granularity,'Year']).sum()
    
    #create vt and ct percentage column
    df_copy['vt_pctg'] = df_copy['all_conversions_vt'] / df_copy['total_conversions']
    df_copy['ct_pctg'] = df_copy['all_conversions_ct'] / df_copy['total_conversions']

    #fix nulls in vt and ct pctg col
    df_copy.loc[(df_copy['total_conversions'] == 0), 'vt_pctg'] = 0
    df_copy.loc[(df_copy['total_conversions'] == 0), 'ct_pctg'] = 0

    #create average ct column
    df_copy['avg_ct_pctg'] = df_copy['ct_pctg'].mean()

    #edge case -> when ct_pctg == 0, set multiplier to null rather than inf
    def safe_divide(a,b):
        if b and b != 0 and b != np.nan: 
            return a/b
        else: 
            return None
        
    #create individual mult for each row and avg multiplier across all rows
    df_copy['indiv_multiplier'] = df_copy.apply(lambda x: safe_divide(x['vt_pctg'],x['ct_pctg']),axis=1)
    df_copy['avg_multiplier'] = df_copy['indiv_multiplier'].mean()
    
    return df_copy,df_copy['indiv_multiplier'].mean()

## Eval Error Function

In [429]:
'''
    FIX THIS to match eval error 
    purpose
    --------
        Calculate a multiplier that uses click through conversions to project view through conversions.
        The multiplier calculates vt conversions using ct conversions.
    arguments
    ---------
        df : dataframe
            raw BQ data formatted based on the above query 
        date_granularity : str (exact column name)
            level of granularity when grouping the data to calculate a multiplier 
        start_date : datetime
            training start date (default 2020-01-01)
        end_date : datetime
            training end date (default 2020-12-31)
    returns
    -------
        df_copy: dataframe
            dataframe with multiplier 
            [0]
        df_copy['indiv_multiplier'].mean() : int
            the multiplier number itself 
            [1]
'''


def eval_error(df, date_granularity,start_date='2020-01-01',end_date='2020-12-31'):
    #MAKE A COPY OF DF!!!
    df_full = df
    
    #get multiplier
    df_train = calculate_multiplier(df_full,date_granularity,start_date,end_date)[0]
    mult = calculate_multiplier(df_full,date_granularity,start_date,end_date)[1]
    
    #group 
    df_full = df_full.groupby([date_granularity,'Year']).sum()
    
    #create sample col in full df and label in-sample vs out_samp
    #not working bc date_granularity isn't an actual column, so I used Quarter instead
    df_full['sample'] = 'in samp'
    in_sample_list = df_train['Quarter'].tolist()
    df_full.loc[~df_full['Quarter'].isin(in_sample_list), 'sample'] = 'out_samp'
  
    #try another method
#     df_full['sample'] = 'in samp'
#     df_out_samp = pd.concat([df_full,df_train]).drop_duplicates(keep=False)
#     return df_out_samp.shape, df_train.shape, df_full.shape
#     #df_full['sample'].loc[df_out_samp['sample'] == 'out samp'
    
    #create out_samp df 
    df_out_samp = df_full[df_full['sample'] == 'out_samp']
    df_out_samp['projected'] = df_out_samp['all_conversions_ct'] * mult
    df_out_samp['abs_error'] = abs(df_out_samp['projected'] - df_out_samp['all_conversions_vt'])
    
    #edge case -> set error to 1 when vt conversions == 0 but we predicted > 0 
    #no null in if statement bc vt conversions are never null
    def safe_divide(a,b):
        if b and b != 0:
            return a/b
        else: 
            return 1
    
    #calculate Absolute Error Percentage and the Mean Absolute Percentage Error
    df_out_samp['abs_error_pctg'] = df_out_samp.apply(lambda x: safe_divide(x['abs_error'],x['all_conversions_vt']),axis=1)
    df_out_samp['MAE'] = df_out_samp['abs_error'].mean()
    df_out_samp['MAPE'] = df_out_samp['abs_error_pctg'].mean()
    
    return df_out_samp,df_out_samp['abs_error'].mean(),df_out_samp['abs_error_pctg'].mean()

# Calculating Multipliers (Split by Channel, Aggregated by Month)

## Display Multiplier -> 153.30

In [430]:
df_display.shape

(244309, 41)

In [431]:
calculate_multiplier(df_display,'year_month_cal')[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,vt_pctg,ct_pctg,avg_ct_pctg,indiv_multiplier,avg_multiplier
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,2020,13542,13542,28511,1.710893e+06,165127.0,408536870.0,293.0,15.0,10.0,15.0,7.0,318.0,340.0,11390.0,1088.0,1278.0,439.0,1996.0,13107.0,16191.0,16531.0,0.979433,0.020567,0.013044,47.620588,79.297108
2020-02-01,2020,12952,25904,83469,1.218100e+06,101148.0,289012941.0,171.0,5.0,6.0,14.0,4.0,191.0,200.0,13515.0,1157.0,1764.0,370.0,1760.0,15649.0,18566.0,18766.0,0.989342,0.010658,0.013044,92.830000,79.297108
2020-03-01,2020,14696,44088,156104,1.822420e+06,160781.0,423888680.0,302.0,13.0,11.0,18.0,6.0,331.0,350.0,26660.0,1546.0,1604.0,766.0,2800.0,29030.0,33376.0,33726.0,0.989622,0.010378,0.013044,95.360000,79.297108
2020-04-01,2020,26392,52784,200904,1.112285e+06,169992.0,524199966.0,403.0,18.0,7.0,53.0,9.0,463.0,490.0,24992.0,2281.0,1553.0,1121.0,4508.0,27666.0,34455.0,34945.0,0.985978,0.014022,0.013044,70.316327,79.297108
2020-05-01,2020,5708,14270,50948,6.120296e+04,7877.0,22561763.0,43.0,14.0,3.0,8.0,4.0,54.0,72.0,3058.0,916.0,183.0,270.0,923.0,3511.0,5350.0,5422.0,0.986721,0.013279,0.013044,74.305556,79.297108
2020-07-01,2020,32214,75166,302784,7.246330e+05,121368.0,227849885.0,235.0,10.0,5.0,18.0,2.0,258.0,270.0,14203.0,1199.0,613.0,539.0,1422.0,15355.0,17976.0,18246.0,0.985202,0.014798,0.013044,66.577778,79.297108
2020-08-01,2020,35769,95384,386682,7.129080e+05,108814.0,222860413.0,162.0,5.0,5.0,11.0,2.0,178.0,185.0,11389.0,851.0,572.0,530.0,1640.0,12491.0,14982.0,15167.0,0.987802,0.012198,0.013044,80.983784,79.297108
2020-09-01,2020,37782,113346,463901,8.611099e+05,100621.0,233283896.0,164.0,7.0,3.0,26.0,1.0,193.0,201.0,13366.0,757.0,815.0,616.0,1715.0,14797.0,17269.0,17470.0,0.988495,0.011505,0.013044,85.915423,79.297108
2020-10-01,2020,74920,187300,768793,1.527231e+06,152109.0,348563450.0,194.0,5.0,4.0,37.0,5.0,235.0,245.0,14345.0,1361.0,953.0,764.0,2068.0,16062.0,19491.0,19736.0,0.987586,0.012414,0.013044,79.555102,79.297108


In [432]:
calculate_multiplier(df_display,'year_month_cal')[1]

79.29710847027712

In [441]:
display_error = eval_error(df_display, 'year_month_cal')
display_error[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,sample,projected,abs_error,abs_error_pctg,MAE,MAPE
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01,2021,17688,17688,37658,2.067065e+06,237307.0,505547231.0,284.0,10.0,16.0,57.0,6.0,357.0,373.0,17218.0,1613.0,830.0,977.0,2940.0,19025.0,23578.0,23951.0,out_samp,29577.821459,5999.821459,0.254467,3176.205705,0.271422
2021-02-01,2021,19974,39948,131012,1.387021e+06,134135.0,307301306.0,82.0,9.0,0.0,33.0,11.0,115.0,135.0,7386.0,1395.0,542.0,559.0,2198.0,8487.0,12080.0,12215.0,out_samp,10705.109643,1374.890357,0.113815,3176.205705,0.271422
2021-03-01,2021,18943,56829,199571,1.051338e+06,83881.0,236094603.0,53.0,10.0,0.0,28.0,2.0,81.0,93.0,6127.0,1132.0,390.0,524.0,1498.0,7041.0,9671.0,9764.0,out_samp,7374.631088,2296.368912,0.237449,3176.205705,0.271422
2021-04-01,2021,29362,58724,222958,8.485122e+05,71345.0,162992545.0,66.0,6.0,1.0,17.0,5.0,84.0,95.0,4215.0,843.0,222.0,261.0,1010.0,4698.0,6551.0,6646.0,out_samp,7533.225305,982.225305,0.149935,3176.205705,0.271422
2021-05-01,2021,28260,70650,277205,1.117045e+06,98882.0,211415709.0,178.0,5.0,8.0,17.0,6.0,203.0,214.0,6848.0,854.0,250.0,333.0,1243.0,7431.0,9528.0,9742.0,out_samp,16969.581213,7441.581213,0.781022,3176.205705,0.271422
2021-06-01,2021,29160,87480,348029,1.885979e+06,115716.0,246271674.0,102.0,2.0,2.0,10.0,4.0,114.0,120.0,8115.0,630.0,238.0,303.0,1192.0,8656.0,10478.0,10598.0,out_samp,9515.653016,962.346984,0.091845,3176.205705,0.271422


In [442]:
display_error[1]

3176.2057048454267

In [443]:
display_error[2]

0.27142223416160177

## Native Multiplier 

In [444]:
df_native.shape

(233033, 41)

In [445]:
calculate_multiplier(df_native,'year_month_cal')[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,vt_pctg,ct_pctg,avg_ct_pctg,indiv_multiplier,avg_multiplier
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,2020,15440,15440,33083,1.307507e+06,6179954.0,330761943.0,637.0,17.0,58.0,7.0,30.0,702.0,749.0,5226.0,304.0,728.0,242.0,694.0,6196.0,7194.0,7943.0,0.905703,0.094297,0.025065,9.604806,63.054055
2020-02-01,2020,19739,39478,128822,1.022491e+06,2918944.0,286138229.0,351.0,19.0,35.0,6.0,36.0,392.0,447.0,6227.0,270.0,808.0,168.0,551.0,7203.0,8024.0,8471.0,0.947232,0.052768,0.025065,17.950783,63.054055
2020-03-01,2020,12321,36963,120760,7.400668e+05,125972.0,190761084.0,116.0,2.0,3.0,9.0,2.0,128.0,132.0,7966.0,193.0,611.0,202.0,403.0,8779.0,9375.0,9507.0,0.986115,0.013885,0.025065,71.022727,63.054055
2020-04-01,2020,3620,7240,26878,3.181985e+05,77591.0,180313886.0,130.0,3.0,3.0,11.0,3.0,144.0,150.0,7908.0,244.0,548.0,321.0,783.0,8777.0,9804.0,9954.0,0.984931,0.015069,0.025065,65.360000,63.054055
2020-05-01,2020,628,1570,5603,6.534766e+03,3095.0,4185552.0,9.0,0.0,0.0,1.0,0.0,10.0,10.0,901.0,67.0,52.0,81.0,188.0,1034.0,1289.0,1299.0,0.992302,0.007698,0.025065,128.900000,63.054055
2020-07-01,2020,16266,37954,156960,2.656784e+05,56373.0,114984774.0,52.0,0.0,0.0,8.0,2.0,60.0,62.0,4479.0,103.0,158.0,128.0,284.0,4765.0,5152.0,5214.0,0.988109,0.011891,0.025065,83.096774,63.054055
2020-08-01,2020,45810,122160,496683,7.009453e+05,148332.0,239214568.0,129.0,6.0,0.0,6.0,7.0,135.0,148.0,6160.0,227.0,303.0,231.0,664.0,6694.0,7585.0,7733.0,0.980861,0.019139,0.025065,51.250000,63.054055
2020-09-01,2020,46194,138582,568649,7.935077e+05,133674.0,238724809.0,133.0,1.0,2.0,5.0,7.0,140.0,148.0,7137.0,218.0,407.0,300.0,646.0,7844.0,8708.0,8856.0,0.983288,0.016712,0.025065,58.837838,63.054055
2020-10-01,2020,67700,169250,697826,1.089367e+06,176535.0,262422195.0,123.0,1.0,5.0,12.0,3.0,140.0,144.0,6677.0,245.0,363.0,278.0,716.0,7318.0,8279.0,8423.0,0.982904,0.017096,0.025065,57.493056,63.054055


In [446]:
calculate_multiplier(df_native,'year_month_cal')[1]

63.05405537515563

In [451]:
native_error = eval_error(df_native, 'year_month_cal')
native_error[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,sample,projected,abs_error,abs_error_pctg,MAE,MAPE
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01,2021,13061,13061,31343,1.528667e+06,365915.0,237883709.0,208.0,2.0,1.0,11.0,16.0,220.0,238.0,8095.0,284.0,357.0,320.0,961.0,8772.0,10017.0,10255.0,out_samp,15006.865179,4989.865179,0.498140,1211.342391,0.192408
2021-02-01,2021,12386,24772,81518,5.919916e+05,120236.0,85062479.0,37.0,0.0,0.0,2.0,3.0,39.0,42.0,2462.0,163.0,148.0,121.0,503.0,2731.0,3397.0,3439.0,out_samp,2648.270326,748.729674,0.220409,1211.342391,0.192408
2021-03-01,2021,16345,49035,180157,3.354345e+05,59429.0,46008018.0,32.0,0.0,0.0,1.0,1.0,33.0,34.0,1855.0,127.0,113.0,65.0,274.0,2033.0,2434.0,2468.0,out_samp,2143.837883,290.162117,0.119212,1211.342391,0.192408
2021-04-01,2021,38186,76372,289985,5.485935e+05,95298.0,64430573.0,37.0,1.0,3.0,4.0,2.0,44.0,47.0,2209.0,107.0,92.0,89.0,221.0,2390.0,2718.0,2765.0,out_samp,2963.540603,245.540603,0.090339,1211.342391,0.192408
2021-05-01,2021,38892,97230,380881,8.053040e+05,117863.0,111570752.0,52.0,0.0,0.0,4.0,2.0,56.0,58.0,3602.0,203.0,109.0,139.0,364.0,3850.0,4417.0,4475.0,out_samp,3657.135212,759.864788,0.172032,1211.342391,0.192408
2021-06-01,2021,40996,122988,491646,9.956575e+05,137095.0,119626650.0,56.0,8.0,2.0,4.0,2.0,62.0,72.0,3576.0,193.0,98.0,136.0,303.0,3810.0,4306.0,4378.0,out_samp,4539.891987,233.891987,0.054318,1211.342391,0.192408


In [452]:
native_error[1]

1211.342391443284

In [453]:
native_error[2]

0.19240818122185943

## Display and Native Multiplier

In [454]:
df_display_native.shape

(477342, 41)

In [455]:
calculate_multiplier(df_display_native,'year_month_cal')[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,vt_pctg,ct_pctg,avg_ct_pctg,indiv_multiplier,avg_multiplier
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,2020,28982,28982,61594,3.018400e+06,6345081.0,739298813.0,930.0,32.0,68.0,22.0,37.0,1020.0,1089.0,16616.0,1392.0,2006.0,681.0,2690.0,19303.0,23385.0,24474.0,0.955504,0.044496,0.016938,21.473829,68.500662
2020-02-01,2020,32691,65382,212291,2.240590e+06,3020092.0,575151170.0,522.0,24.0,41.0,20.0,40.0,583.0,647.0,19742.0,1427.0,2572.0,538.0,2311.0,22852.0,26590.0,27237.0,0.976246,0.023754,0.016938,41.097372,68.500662
2020-03-01,2020,27017,81051,276864,2.562487e+06,286753.0,614649764.0,418.0,15.0,14.0,27.0,8.0,459.0,482.0,34626.0,1739.0,2215.0,968.0,3203.0,37809.0,42751.0,43233.0,0.988851,0.011149,0.016938,88.695021,68.500662
2020-04-01,2020,30012,60024,227782,1.430483e+06,247583.0,704513852.0,533.0,21.0,10.0,64.0,12.0,607.0,640.0,32900.0,2525.0,2101.0,1442.0,5291.0,36443.0,44259.0,44899.0,0.985746,0.014254,0.016938,69.154688,68.500662
2020-05-01,2020,6336,15840,56551,6.773772e+04,10972.0,26747315.0,52.0,14.0,3.0,9.0,4.0,64.0,82.0,3959.0,983.0,235.0,351.0,1111.0,4545.0,6639.0,6721.0,0.987799,0.012201,0.016938,80.963415,68.500662
2020-07-01,2020,48480,113120,459744,9.903115e+05,177741.0,342834659.0,287.0,10.0,5.0,26.0,4.0,318.0,332.0,18682.0,1302.0,771.0,667.0,1706.0,20120.0,23128.0,23460.0,0.985848,0.014152,0.016938,69.662651,68.500662
2020-08-01,2020,81579,217544,883365,1.413853e+06,257146.0,462074981.0,291.0,11.0,5.0,17.0,9.0,313.0,333.0,17549.0,1078.0,875.0,761.0,2304.0,19185.0,22567.0,22900.0,0.985459,0.014541,0.016938,67.768769,68.500662
2020-09-01,2020,83976,251928,1032550,1.654618e+06,234295.0,472008705.0,297.0,8.0,5.0,31.0,8.0,333.0,349.0,20503.0,975.0,1222.0,916.0,2361.0,22641.0,25977.0,26326.0,0.986743,0.013257,0.016938,74.432665,68.500662
2020-10-01,2020,142620,356550,1466619,2.616599e+06,328644.0,610985645.0,317.0,6.0,9.0,49.0,8.0,375.0,389.0,21022.0,1606.0,1316.0,1042.0,2784.0,23380.0,27770.0,28159.0,0.986186,0.013814,0.016938,71.388175,68.500662


In [456]:
calculate_multiplier(df_display_native,'year_month_cal')[1]

68.50066204031445

In [457]:
display_native_error = eval_error(df_display_native, 'year_month_cal')
display_native_error[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,sample,projected,abs_error,abs_error_pctg,MAE,MAPE
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01,2021,30749,30749,69001,3.595732e+06,603222.0,743430940.0,492.0,12.0,17.0,68.0,22.0,577.0,611.0,25313.0,1897.0,1187.0,1297.0,3901.0,27797.0,33595.0,34206.0,out_samp,41853.904507,8258.904507,0.245837,3632.30837,0.206614
2021-02-01,2021,32360,64720,212530,1.979013e+06,254371.0,392363785.0,119.0,9.0,0.0,35.0,14.0,154.0,177.0,9848.0,1558.0,690.0,680.0,2701.0,11218.0,15477.0,15654.0,out_samp,12124.617181,3352.382819,0.216604,3632.30837,0.206614
2021-03-01,2021,35288,105864,379728,1.386773e+06,143310.0,282102621.0,85.0,10.0,0.0,29.0,3.0,114.0,127.0,7982.0,1259.0,503.0,589.0,1772.0,9074.0,12105.0,12232.0,out_samp,8699.584079,3405.415921,0.281323,3632.30837,0.206614
2021-04-01,2021,67548,135096,512943,1.397106e+06,166643.0,227423118.0,103.0,7.0,4.0,21.0,7.0,128.0,142.0,6424.0,950.0,314.0,350.0,1231.0,7088.0,9269.0,9411.0,out_samp,9727.094010,458.094010,0.049422,3632.30837,0.206614
2021-05-01,2021,67152,167880,658086,1.922349e+06,216745.0,322986461.0,230.0,5.0,8.0,21.0,8.0,259.0,272.0,10450.0,1057.0,359.0,472.0,1607.0,11281.0,13945.0,14217.0,out_samp,18632.180075,4687.180075,0.336119,3632.30837,0.206614
2021-06-01,2021,70156,210468,839675,2.881636e+06,252811.0,365898324.0,158.0,10.0,4.0,14.0,6.0,176.0,192.0,11691.0,823.0,336.0,439.0,1495.0,12466.0,14784.0,14976.0,out_samp,13152.127112,1631.872888,0.110381,3632.30837,0.206614


In [458]:
display_native_error = eval_error(df_display_native, 'year_month_cal')
display_native_error[1]

3632.3083698877235

In [459]:
display_native_error = eval_error(df_display_native, 'year_month_cal')
display_native_error[2]

0.20661446473439493

## Social Multiplier

In [460]:
df_social.shape

(101599, 41)

In [461]:
calculate_multiplier(df_social,'year_month_cal')[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,vt_pctg,ct_pctg,avg_ct_pctg,indiv_multiplier,avg_multiplier
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-01,2020,8469,8469,20402,6.087191e+06,10132146.0,1.414825e+09,19795.0,127.0,0.0,620.0,1235.0,22941.0,24303.0,22866.0,326.0,1.0,881.0,855.0,39100.0,40281.0,64584.0,0.623699,0.376301,0.263528,1.657450,3.066821
2020-02-01,2020,9717,19434,62405,4.391414e+06,5424796.0,7.893624e+08,12737.0,33.0,0.0,445.0,438.0,15205.0,15676.0,18661.0,266.0,0.0,563.0,586.0,31332.0,32184.0,47860.0,0.672461,0.327539,0.263528,2.053075,3.066821
2020-03-01,2020,7513,22539,79648,4.590198e+06,4353516.0,9.974523e+08,11783.0,138.0,0.0,733.0,1044.0,15148.0,16330.0,44252.0,554.0,0.0,1593.0,1248.0,65706.0,67508.0,83838.0,0.805220,0.194780,0.263528,4.133987,3.066821
2020-04-01,2020,14936,29872,113931,2.657055e+06,2942695.0,7.783464e+08,9286.0,271.0,0.0,956.0,2314.0,11993.0,14578.0,33192.0,841.0,0.0,1688.0,2121.0,50995.0,53957.0,68535.0,0.787291,0.212709,0.263528,3.701262,3.066821
2020-05-01,2020,3866,9665,34679,1.307054e+05,184859.0,3.717231e+07,1217.0,58.0,0.0,87.0,383.0,1452.0,1893.0,3203.0,336.0,0.0,209.0,566.0,4167.0,5069.0,6962.0,0.728095,0.271905,0.263528,2.677760,3.066821
2020-07-01,2020,7311,17059,70068,1.589868e+06,1137856.0,3.846130e+08,4393.0,0.0,0.0,223.0,0.0,5111.0,5111.0,14092.0,0.0,0.0,773.0,0.0,24808.0,24808.0,29919.0,0.829172,0.170828,0.263528,4.853845,3.066821
2020-08-01,2020,10071,26856,110845,1.742489e+06,1244540.0,3.589467e+08,3940.0,0.0,0.0,213.0,0.0,4476.0,4476.0,14015.0,0.0,0.0,875.0,0.0,22572.0,22572.0,27048.0,0.834516,0.165484,0.263528,5.042895,3.066821
2020-09-01,2020,11037,33111,137728,2.391346e+06,1898981.0,2.990519e+08,8372.0,0.0,0.0,345.0,0.0,9071.0,9071.0,16572.0,0.0,0.0,869.0,0.0,24618.0,24618.0,33689.0,0.730743,0.269257,0.263528,2.713923,3.066821
2020-10-01,2020,20792,51980,216248,2.803931e+06,2018756.0,3.789039e+08,9058.0,39.0,0.0,394.0,408.0,9927.0,10374.0,14276.0,116.0,0.0,857.0,179.0,22124.0,22419.0,32793.0,0.683652,0.316348,0.263528,2.161076,3.066821


In [462]:
calculate_multiplier(df_social,'year_month_cal')[1]

3.066821238216359

In [463]:
social_error = eval_error(df_social, 'year_month_cal')
social_error[0]

,,Quarter,Month,week,spend,clicks,impressions,digi_ada_conversions_ct,games_conversions_ct,edu_conversions_ct,hd_conversions_ct,ck_conversions_ct,core_conversions_ct,all_conversions_ct,digi_ada_conversions_vt,games_conversions_vt,edu_conversions_vt,hd_conversions_vt,ck_conversions_vt,core_conversions_vt,all_conversions_vt,total_conversions,sample,projected,abs_error,abs_error_pctg,MAE,MAPE
year_month_cal,Year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01,2021,3610,3610,10603,3.287436e+06,2254194.0,533715728.0,13734.0,0.0,0.0,691.0,0.0,15034.0,15034.0,26861.0,0.0,0.0,1724.0,0.0,40472.0,40472.0,55506.0,out_samp,46106.590495,5634.590495,0.139222,4859.34961,0.279204
2021-02-01,2021,4989,9978,33582,1.829516e+06,1464169.0,291625109.0,5422.0,406.0,0.0,396.0,968.0,6044.0,7418.0,9161.0,1038.0,0.0,645.0,2181.0,14142.0,17361.0,24779.0,out_samp,22749.679945,5388.679945,0.310390,4859.34961,0.279204
2021-03-01,2021,5136,15408,55882,1.375490e+06,1357774.0,198061913.0,4389.0,356.0,0.0,285.0,688.0,4860.0,5904.0,6952.0,812.0,0.0,426.0,1280.0,10130.0,12222.0,18126.0,out_samp,18106.512590,5884.512590,0.481469,4859.34961,0.279204
2021-04-01,2021,12692,25384,98539,1.237896e+06,1031961.0,188777143.0,4330.0,249.0,0.0,267.0,710.0,4778.0,5737.0,7973.0,832.0,0.0,510.0,1144.0,11348.0,13324.0,19061.0,out_samp,17594.353444,4270.353444,0.320501,4859.34961,0.279204
2021-05-01,2021,9832,24580,95521,2.699790e+06,1886402.0,350880219.0,6630.0,385.0,0.0,403.0,691.0,7331.0,8407.0,10663.0,1121.0,0.0,659.0,1523.0,16260.0,18904.0,27311.0,out_samp,25782.766150,6878.766150,0.363879,4859.34961,0.279204
2021-06-01,2021,9488,28464,113874,3.320755e+06,2149873.0,412926005.0,4052.0,253.0,0.0,263.0,552.0,4834.0,5639.0,9197.0,877.0,0.0,479.0,1133.0,16383.0,18393.0,24032.0,out_samp,17293.804962,1099.195038,0.059762,4859.34961,0.279204


In [464]:
social_error = eval_error(df_social, 'year_month_cal')
social_error[1]

4859.349610315537

In [465]:
social_error = eval_error(df_social, 'year_month_cal')
social_error[2]

0.2792036823903637

## Facebook Multiplier

In [ ]:
df_fb_removed = removal(df_fb)

In [ ]:
df_fb.shape

In [ ]:
df_fb_removed.shape

In [ ]:
actual_multiplier(df_fb_removed)

In [ ]:
df_multiplier(df_fb_removed)

## Snap Multiplier

In [ ]:
df_snap_removed = removal(df_snap)

In [ ]:
df_snap.shape

In [ ]:
df_snap_removed.shape

In [ ]:
actual_multiplier(df_snap_removed)

In [ ]:
df_multiplier(df_snap_removed)

## Twitter Multiplier

In [ ]:
df_twitter_removed = removal(df_twitter)

In [ ]:
df_twitter.shape

In [ ]:
df_twitter_removed.shape

In [ ]:
actual_multiplier(df_twitter_removed)

In [ ]:
df_multiplier(df_twitter_removed)

# Data Cleaning Section at Top

get rid of Paid Search, spend 0, combine low-volume channels, sites, etc.
centralize and consolidate -> make adjustments before splitting into channel dfs 

# Functions

make more dynamic
training range 2020, out of sample -> 2021 (could potentially adjust)
run the actual functions and run for each channel (at each granularity) 
and get % error table

# Appendix

In [ ]:
# Attempts to group high vol and low vol (that didn't rly work)


#group low-volume channels, products, and sites, and split out channel-Social
# def new_channel(channel):
#     if (channel == 'Social') | (channel == 'Display') | (channel == 'Paid Search') | (channel == 'Native'):
#         return channel
#     else:
#         return 'Other'
    
# # for x in df_copy['channel']:
# #   print(new_channel(x))
    
# df_copy['new_channel'] = df_copy.apply(lambda x: new_channel(x['channel'],axis=1))
#~ -> not

In [ ]:
#REMOVAL FUNCTION -> Changed 7.12.21 to manually/static/permanent remove at top
# #returns df with outliers, low-vol channels, sites, and products removed
# # pretty much don't need -> move elsewhere 

# #impact of removals: 

# #if ALL (outliers, paid search, all 3 low vol) removed, then the df is cut in more than half (832638 -> 374316)
# #Outliers (702,551)
# #Paid Search (657,082)
# #Total Conversions = 0 (613,001)
# #Low-volume channels (772,840)
# #low-volume sites (831,519)
# #low-volume products (754,941)
# #all 3 low-volume (733,921)

# # some of it move to the top and remove once -> Paid Search
# # channels where there is 0 spend -> remove it for each specific dataframe 

# #def removal(df):
# #     #remove_outliers
# #     df_spend = df
# #     q1, q3 = np.percentile(df_spend['spend'], [25, 75])
# #     iqr = q3 - q1
# #     lower_bound = q1 - (iqr * 1.5)
# #     upper_bound = q3 + (iqr * 1.5)
# #     df_outliers_remov= df[df['spend'] > lower_bound][df['spend'] < upper_bound] 

#     # df_spend['spend'].describe(percentile = )
    
#     #remove Paid Search
#     df_paid_search_remov = df_outliers_remov.loc[df['channel'] != 'Paid Search']
    
# #     #remove low-volume channels
# #     df_high_vol_channel = df_paid_search_remov.loc[df['new_channel'] != 'Low-Volume Channels']
    
# #     #remove low-volume sites
# #     df_high_vol_sites = df_high_vol_channel.loc[df['new_site'] != 'Low-Volume Sites']
    
# #     #remove low-volume products
# #     df_high_vol_products = df_high_vol_sites.loc[df['new_product'] != 'Low-Volume Products']
    
#     #remove any June 2020 for Native -> infinity (spend = 0)
#     df_remov_native_zero = df_high_vol_products.loc[~((df['year_month_cal'] == '2020-06-01') & 
#                                              (df['channel'] == 'Native'))]
    
#     #remove any June 2020 for Twitter -> infinity (spend = 0)
#     df_remov_twitter_zero = df_remov_native_zero.loc[~((df['year_month_cal'] == '2020-06-01') & 
#                                              (df['channel'] == 'Social') &
#                                              (df['site'] == 'Twitter'))]
#     #remove total conversions = 0 
#     df_final_remov = df_remov_twitter_zero.loc[df['total_conversions'] == 0]
    
#     return df_final_remov

In [ ]:
 #drop cols you don't want in solution attempt -> maybe don't need to drop them 
#     df_copy = df_copy.drop(['account','campaign','_match','campaign_objective','Month','Quarter','week','week_start','date','marketing_initiative',
#               'marketing_segment','platform',
#              'digi_ada_conversions_ct','games_conversions_ct','edu_conversions_ct','hd_conversions_ct','ck_conversions_ct','core_conversions_ct',
#              'digi_ada_conversions_vt','games_conversions_vt','edu_conversions_vt','hd_conversions_vt','ck_conversions_vt','core_conversions_vt',
#              'year_week_monday','channel_mkt_init','new_channel','new_site','new_product'], axis=1)
#    df_copy.reset_index(inplace = True)

# Next Steps

2020 training data, make table, can think about changing training range depending on error